In [12]:
## Run selenium and chrome driver to scrape data from cloudbytes.dev
import time
import os.path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import chromedriver_autoinstaller
import sqlite3
import re

In [13]:
sample = pd.read_csv('AO3/ao3_lockwood_and_co_ao_20230616_0824.csv')
query = "SELECT * FROM fanfic"
conn = sqlite3.connect('output/ao3.db')
df = pd.read_sql_query(query, conn)
query = "SELECT * FROM authors"
df_author = pd.read_sql_query(query, conn)
query = "SELECT * FROM fandoms"
df_fandom = pd.read_sql_query(query, conn)
conn.close()
print(df_author)

    author_id                     author  joined_ao3
0     2889062                      McKat  2017-03-06
1    11032043                  ersatzist  2021-05-11
2    13764538               mayfloweruru  2022-03-22
3    14142288                  crow_blue  2022-05-12
4    16848577              tvngerine_zso  2023-03-14
..        ...                        ...         ...
663   6561073                 SweeterEnd  2020-01-07
664    574299  WindsAroundATreeLikeAVine  2014-07-26
665  14696172       Writing_random_stuff  2022-07-20
666   1711910              Amazinggrace0  2015-12-22
667  14282802              goldenjupiter  2022-05-31

[668 rows x 3 columns]


In [20]:
def get_authors(authors, fandoms):
    for x in range(len(authors)):
        #print(f"getting author {x+1}/{len(authors)}")
        if pd.isnull(authors.loc[x,'author_id']):
            author_orig = authors.loc[x,'author']
            matches = re.findall(r'\((.*?)\)', author_orig)
            if matches:
                author_orig = matches[0]
            elif author_orig=="jhyxn":
                author_orig='suedive'
            elif author_orig=='jwrites23':
                author_orig='rowena_soreland'
            print(f"getting author {x+1}/{len(authors)}")
            try:
                newlink=f"https://archiveofourown.org/users/{author_orig}/profile"
                profile = requests.get(newlink, headers={
                          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}).text
                profile = BeautifulSoup(profile,'html.parser')
                try:
                    pseuds = profile.find('dl', attrs={'class':'meta'})
                    pseuds = pseuds.find_all('dd')
                    joined = pseuds[1].get_text()
                    id = pseuds[2].get_text()
                except:
                    joined = np.nan
                    id = np.nan
        
                authors.loc[x,'author_id'] = id
                authors.loc[x,'joined_ao3'] = joined
        
                time.sleep(10)
            except requests.exceptions.RequestException:
                print(f"author {authors['author'][x]} fandom is taking too long to access.")
                continue
           
            try:
                newlink=f"https://archiveofourown.org/users/{author_orig}"
                user = requests.get(newlink, headers={
                          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}).text
                user = BeautifulSoup(user,'html.parser')
                try:
                    index = user.find('ol', attrs={'class':'index group'})
                    index = index.find_all('li')
                    for i in range(len(index)):
                        li = index[i].get_text().split('(')
                        fandom = li[0].strip()
                        num_fic = li[-1].split(')')[0]
                        fandoms.loc[len(fandoms)] = [id, fandom, num_fic]
                except:
                    print(f"author {authors['author'][x]} fandom is taking too long to access.")
                time.sleep(10)
            except requests.exceptions.RequestException:
                print(f"author {authors['author'][x]} profile is taking too long to access.")
                continue
            
    return authors, fandoms
authors, fandoms = get_authors(df_author, df_fandom)

getting author 35/668
getting author 59/668
getting author 385/668
getting author 437/668
getting author 496/668
author CharmQuartStrangeQuark fandom is taking too long to access.
getting author 507/668
author GabyU07 fandom is taking too long to access.
getting author 662/668


In [21]:
df_author[df_author['author_id'].isna()]

,author_id,author,joined_ao3
34,NaN,loonyloopyluna (orphan_account),NaN
58,NaN,orphan_account,NaN
384,NaN,ceta346 (orphan_account),NaN
495,NaN,CharmQuartStrangeQuark,NaN
506,NaN,GabyU07,NaN


In [ ]:
query = "SELECT * FROM fanfic"
conn = sqlite3.connect('output/ao3.db')
df_author = pd.read_sql_query(query, conn)
print(df_author)
conn.close()

     fanfic_id                                        link  \
0     44791411  https://archiveofourown.org/works/44791411   
1     47484628  https://archiveofourown.org/works/47484628   
2     45235999  https://archiveofourown.org/works/45235999   
3     45056584  https://archiveofourown.org/works/45056584   
4     47517634  https://archiveofourown.org/works/47517634   
...        ...                                         ...   
2042  47837212  https://archiveofourown.org/works/47837212   
2043  47802628  https://archiveofourown.org/works/47802628   
2044  45773677  https://archiveofourown.org/works/45773677   
2045  46481659  https://archiveofourown.org/works/46481659   
2046  47821801  https://archiveofourown.org/works/47821801   

                                                  title   published language  \
0                                           Easily done  2023-02-04  English   
1                                      I will trust you  2023-05-28  English   
2              

In [23]:
query = '''
    SELECT *
    FROM fanfic
'''
conn = sqlite3.connect('output/ao3.db')
df = pd.read_sql_query(query, conn)
conn.close()
sample = pd.read_csv('AO3/ao3_lockwood_and_co_ao_20230616_0824.csv')
sample.columns

Index(['link', 'title', 'author', 'published', 'updatedate', 'chapters',
       'language', 'words', 'kudos', 'comments', 'bookmarks', 'hits',
       'warning', 'relationship', 'characters', 'tags', 'summary', 'rating',
       'series', 'chapter', 'chapter_max', 'completion', 'currentdate',
       'datediff_pub', 'datediff', 'classification', 'lastauthorupdate',
       'firstauthorupdate', 'author_lastupdate_diff', 'daysactive',
       'daysincefirtupload', 'author_activity', 'num_relationship',
       'num_characters', 'num_tags', 'prev_words', 'prev_hits', 'prev_kudos',
       'prev_comments', 'prev_bookmarks'],
      dtype='object')

In [24]:
sample=sample[['link','author']]
sample

,link,author
0,https://archiveofourown.org/works/44791411,The_Biscuit_Agreement
1,https://archiveofourown.org/works/47484628,nemesisn15
2,https://archiveofourown.org/works/45235999,stilesospoon
3,https://archiveofourown.org/works/45056584,Random_Nerd3
4,https://archiveofourown.org/works/47517634,HeavyOnTheTragedy
...,...,...
1997,https://archiveofourown.org/works/3709479,Amelia (BelowTheText)
1998,https://archiveofourown.org/works/2282274,lady_mab
1999,https://archiveofourown.org/works/1267453,lady_mab
2000,https://archiveofourown.org/works/1169828,lady_mab


In [26]:
df =df.drop(columns='author_id')
df = df.merge(sample, how='left', on='link')
df

,fanfic_id,link,title,published,language,summary,warning,author
0,44791411,https://archiveofourown.org/works/44791411,Easily done,2023-02-04,English,"""We are containing the source,"" Lockwood told ...",No Archive Warnings Apply,The_Biscuit_Agreement
1,47484628,https://archiveofourown.org/works/47484628,I will trust you,2023-05-28,English,Lucy and Lockwood running errands and enjoying...,No Archive Warnings Apply,nemesisn15
2,45235999,https://archiveofourown.org/works/45235999,she feels like home,2023-02-22,English,"after waking from a nightmare, Lockwood locks ...",No Archive Warnings Apply,stilesospoon
3,45056584,https://archiveofourown.org/works/45056584,Why So Sad?,2023-02-15,English,“We haven’t seen him since the Thomas case.”“I...,No Archive Warnings Apply,Random_Nerd3
4,47517634,https://archiveofourown.org/works/47517634,It’s a No-Chance Showmance (I Don’t Mix Work W...,2023-05-29,English,"When Kat, who play Audrey in their school’s pr...",Creator Chose Not To Use Archive Warnings,HeavyOnTheTragedy
...,...,...,...,...,...,...,...,...
2074,47837212,https://archiveofourown.org/works/47837212,Flares are expensive,2023-06-12,English,"I said the first thing that came to mind, a re...",Creator Chose Not To Use Archive Warnings,always_unabashed
2075,47802628,https://archiveofourown.org/works/47802628,Let Me Sleep (i am tired of my grief),2023-06-11,English,"""Do you want me to try and move her towards me...",No Archive Warnings Apply,FormallyLateLocklyle
2076,45773677,https://archiveofourown.org/works/45773677,My Rosemary,2023-03-15,English,"Lucy's life had gone quite badly for a while, ...",Graphic Depictions Of Violence,SinSong
2077,46481659,https://archiveofourown.org/works/46481659,I'll be your new favorite tune,2023-04-14,English,"She was a techie, he was an actor. (Can I make...",No Archive Warnings Apply,barronsfever (historyofbellarke)


In [27]:
fanfic = df.merge(authors[['author_id', 'author']], how='left', on='author')
fanfic = fanfic.drop(columns=['author'])
conn = sqlite3.connect('output/ao3.db')
fanfic.to_sql('fanfic', conn, if_exists='replace', index=False)
authors.to_sql('authors', conn, if_exists='replace', index=False)
fandoms.to_sql('fandoms', conn, if_exists='replace', index=False)
conn.close()
fanfic

,fanfic_id,link,title,published,language,summary,warning,author_id
0,44791411,https://archiveofourown.org/works/44791411,Easily done,2023-02-04,English,"""We are containing the source,"" Lockwood told ...",No Archive Warnings Apply,2440594
1,47484628,https://archiveofourown.org/works/47484628,I will trust you,2023-05-28,English,Lucy and Lockwood running errands and enjoying...,No Archive Warnings Apply,15260589
2,45235999,https://archiveofourown.org/works/45235999,she feels like home,2023-02-22,English,"after waking from a nightmare, Lockwood locks ...",No Archive Warnings Apply,8699371
3,45056584,https://archiveofourown.org/works/45056584,Why So Sad?,2023-02-15,English,“We haven’t seen him since the Thomas case.”“I...,No Archive Warnings Apply,3796812
4,47517634,https://archiveofourown.org/works/47517634,It’s a No-Chance Showmance (I Don’t Mix Work W...,2023-05-29,English,"When Kat, who play Audrey in their school’s pr...",Creator Chose Not To Use Archive Warnings,13583521
...,...,...,...,...,...,...,...,...
2074,47837212,https://archiveofourown.org/works/47837212,Flares are expensive,2023-06-12,English,"I said the first thing that came to mind, a re...",Creator Chose Not To Use Archive Warnings,17736727
2075,47802628,https://archiveofourown.org/works/47802628,Let Me Sleep (i am tired of my grief),2023-06-11,English,"""Do you want me to try and move her towards me...",No Archive Warnings Apply,7673995
2076,45773677,https://archiveofourown.org/works/45773677,My Rosemary,2023-03-15,English,"Lucy's life had gone quite badly for a while, ...",Graphic Depictions Of Violence,16404661
2077,46481659,https://archiveofourown.org/works/46481659,I'll be your new favorite tune,2023-04-14,English,"She was a techie, he was an actor. (Can I make...",No Archive Warnings Apply,4046850
